# Notebook 03 — Modeling & Export

**Goal:** Train baseline (LogReg, RF), tune RF, evaluate, and export the best pipeline.  
**Input:** `../data/processed/hr_attrition_ready.parquet`  
**Outputs:** `../artifacts/v1/rf_pipeline.joblib`, `../artifacts/v1/features.json`  


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from pathlib import Path

df = pd.read_parquet("../data/processed/hr_attrition_ready.parquet")

NUM = ["Age","MonthlyIncome","DistanceFromHome","TotalWorkingYears",
       "YearsAtCompany","NumCompaniesWorked","PercentSalaryHike"]
CAT = ["OverTime","JobRole","MaritalStatus","BusinessTravel",
       "Department","EducationField","Gender","JobLevel"]

X = df[NUM + CAT]
y = df["target"]

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

def preproc(num, cat):
    return ColumnTransformer([
        ("num", Pipeline([("impute", SimpleImputer(strategy="median")),
                          ("scale", StandardScaler())]), num),
        ("cat", Pipeline([("impute", SimpleImputer(strategy="most_frequent")),
                          ("onehot", OneHotEncoder(handle_unknown="ignore"))]), cat)
    ])

logreg = Pipeline([("pre", preproc(NUM, CAT)), ("clf", LogisticRegression(max_iter=1000))])
rf     = Pipeline([("pre", preproc(NUM, CAT)), ("clf", RandomForestClassifier(random_state=42))])

for name, model in [("LogReg", logreg), ("RF", rf)]:
    model.fit(Xtr, ytr)
    preds = model.predict(Xte)
    probs = model.predict_proba(Xte)[:,1]
    print(f"\n=== {name} ===")
    print(classification_report(yte, preds))
    print("ROC-AUC:", roc_auc_score(yte, probs))
    print("Confusion matrix:\n", confusion_matrix(yte, preds))


=== LogReg ===
              precision    recall  f1-score   support

           0       0.87      0.97      0.91       247
           1       0.56      0.21      0.31        47

    accuracy                           0.85       294
   macro avg       0.71      0.59      0.61       294
weighted avg       0.82      0.85      0.82       294

ROC-AUC: 0.7768972349039539
Confusion matrix:
 [[239   8]
 [ 37  10]]

=== RF ===
              precision    recall  f1-score   support

           0       0.86      0.96      0.91       247
           1       0.44      0.17      0.25        47

    accuracy                           0.83       294
   macro avg       0.65      0.56      0.58       294
weighted avg       0.79      0.83      0.80       294

ROC-AUC: 0.7623395641312776
Confusion matrix:
 [[237  10]
 [ 39   8]]


In [2]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "clf__n_estimators": [100, 300, 500],
    "clf__max_depth": [None, 8, 16],
    "clf__min_samples_split": [2, 5, 10],
    "clf__min_samples_leaf": [1, 2, 4],
    "clf__max_features": ["sqrt", "log2", None],
    "clf__criterion": ["gini", "entropy", "log_loss"]
}

rf = Pipeline([("pre", preproc(NUM, CAT)), ("clf", RandomForestClassifier(random_state=42))])
grid = GridSearchCV(rf, param_grid=param_grid, scoring="roc_auc", cv=5, n_jobs=-1, verbose=1)
grid.fit(Xtr, ytr)

best = grid.best_estimator_
print("Best params:", grid.best_params_)
from sklearn.metrics import roc_auc_score
print("Test ROC-AUC:", roc_auc_score(yte, best.predict_proba(Xte)[:,1]))

Fitting 5 folds for each of 729 candidates, totalling 3645 fits


Best params: {'clf__criterion': 'gini', 'clf__max_depth': 16, 'clf__max_features': 'sqrt', 'clf__min_samples_leaf': 2, 'clf__min_samples_split': 2, 'clf__n_estimators': 100}
Test ROC-AUC: 0.7682832285295892


In [ ]:
from sklearn.model_selection import GridSearchCV

small_grid = {
    "clf__n_estimators": [150, 300],
    "clf__max_depth": [None, 12],
    "clf__min_samples_split": [2, 5],
    "clf__min_samples_leaf": [1, 2],
    "clf__max_features": ["sqrt", "log2"],
    "clf__criterion": ["gini", "entropy"],
}
rf = Pipeline([("pre", preproc(NUM, CAT)),
               ("clf", RandomForestClassifier(random_state=42))])

grid = GridSearchCV(rf, param_grid=small_grid, scoring="roc_auc", cv=3, n_jobs=-1, verbose=1)
grid.fit(Xtr, ytr)

best = grid.best_estimator_
print("Best params:", grid.best_params_)
from sklearn.metrics import roc_auc_score
print("Test ROC-AUC:", roc_auc_score(yte, best.predict_proba(Xte)[:,1]))

In [3]:
import json, joblib, os
ART = Path("../artifacts/v1"); ART.mkdir(parents=True, exist_ok=True)
joblib.dump(best, ART/"rf_pipeline.joblib")
json.dump(NUM + CAT, open(ART/"features.json", "w"))
print("✅ Exported →", ART/"rf_pipeline.joblib", "and", ART/"features.json")

✅ Exported → ../artifacts/v1/rf_pipeline.joblib and ../artifacts/v1/features.json
